In [18]:
using Flux
using Flux: @epochs, onehotbatch, crossentropy, throttle
using Images
using FileIO
using CSV
using Statistics

train_path = "G:\\Book-Cover-Train\\"
train_csv = "F:\\book-dataset\\Task1\\book30-listing-train.csv"

test_path = "G:\\Book-Cover-Test\\"
test_csv = "F:\\book-dataset\\Task1\\book30-listing-test.csv"

train_imglist = readdir(train_path)
test_imglist = readdir(test_path)

train_setsize = length(train_imglist)
test_setsize = length(test_imglist)

batch_size = 5

5

In [19]:

function create_dataset(indexs; path, csv, images)
    dataset = CSV.read(csv)
    X = Array{Float32}(undef, 100, 100, 3, length(indexs))
    for i = 1:length(indexs)
        img = load(string(path, images[i]))
        img = channelview(imresize(img, 100, 100))
        img = Float32.(permutedims(img, (2, 3, 1)))
        X[:, :, :, i] = img
    end
    Y = onehotbatch(dataset[indexs[1]:indexs[end], 6], 0:29)
    return (X, Y)
end


create_dataset (generic function with 2 methods)

In [20]:
indexs = Base.Iterators.partition(1:train_setsize, batch_size)
train_set = [create_dataset(
    i;
    path = train_path,
    csv = train_csv,
    images = train_imglist,
) for i in indexs]

test_set = create_dataset(
    1:test_setsize;
    path = test_path,
    csv = test_csv,
    images = test_imglist,
)

(Float32[0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.980392 0.964706 … 0.921569 1.0; 1.0 0.996078 … 0.996078 0.952941]

Float32[0.364706 0.364706 … 0.364706 0.364706; 0.364706 0.364706 … 0.364706 0.364706; … ; 0.705882 0.482353 … 0.537255 0.772549; 0.811765 0.819608 … 0.505882 0.513726]

Float32[0.568627 0.568627 … 0.568627 0.568627; 0.568627 0.568627 … 0.568627 0.568627; … ; 0.164706 0.164706 … 0.101961 0.227451; 0.180392 0.192157 … 0.192157 0.219608]

Float32[0.517647 0.517647 … 0.411765 0.388235; 0.52549 0.521569 … 0.411765 0.419608; … ; 0.109804 0.105882 … 0.376471 0.364706; 0.101961 0.0901961 … 0.180392 0.160784]

Float32[0.490196 0.490196 … 0.407843 0.384314; 0.486275 0.486275 … 0.407843 0.415686; … ; 0.14902 0.145098 … 0.380392 0.368627; 0.145098 0.133333 … 0.2 0.192157]

Float32[0.458824 0.45098 … 0.392157 0.368627; 0.45098 0.458824 … 0.392157 0.4; … ; 0.156863 0.152941 … 0.388235 0.376471; 0.152941 0.141176 … 0.211765 0.2]

Float32[0.952941 0.94902 … 0.952941 0.952941; 0.94902

In [14]:

m = Chain(
    Conv((3, 3), 3=>32, pad=(1,1), relu),
    MaxPool((2,2)),

    Conv((3, 3), 32=>64, pad=(1,1), relu),
    MaxPool((2,2)),

    Conv((3, 3), 64=>256, pad=(1,1), relu),
    MaxPool((2,2)),

    Conv((2, 2), 256=>512, pad=(1,1), relu),
    MaxPool((2,2)),

    x -> reshape(x, :, size(x, 4)),
    Dense(18432, 256, relu),
    Dense(256, 30),
    softmax,
    )


Chain(Conv((3, 3), 3=>32, NNlib.relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 32=>64, NNlib.relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((3, 3), 64=>256, NNlib.relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), Conv((2, 2), 256=>512, NNlib.relu), MaxPool((2, 2), pad = (0, 0, 0, 0), stride = (2, 2)), getfield(Main, Symbol("##29#30"))(), Dense(18432, 256, NNlib.relu), Dense(256, 30), NNlib.softmax)

In [15]:
loss(x, y) = crossentropy(m(x), y)
accuracy(x, y) = mean(Flux.onecold(m(x)) .== Flux.onecold(y))

function cbfunc()
    ca = accuracy(test_set...)
    @show(ca)
    cl = loss(test_set...)
    @show(cl)
end

opt = ADAM()
@epochs 5 Flux.train!(loss, params(m), train_set, opt, cb=cbfunc)


┌ Info: Epoch 1
└ @ Main C:\Users\Zubu\.julia\packages\Flux\oX9Pi\src\optimise\train.jl:99


ca = 0.05
cl = 3.4468267f0
ca = 0.06
cl = 3.3383288f0
ca = 0.06
cl = 3.3485308f0
ca = 0.06
cl = 3.387577f0
ca = 0.06
cl = 3.383679f0
ca = 0.06
cl = 3.3617034f0
ca = 0.06
cl = 3.331149f0
ca = 0.09
cl = 3.315382f0
ca = 0.07
cl = 3.3304417f0
ca = 0.07
cl = 3.3429146f0


┌ Info: Epoch 2
└ @ Main C:\Users\Zubu\.julia\packages\Flux\oX9Pi\src\optimise\train.jl:99


ca = 0.07
cl = 3.3417819f0
ca = 0.07
cl = 3.3295848f0
ca = 0.07
cl = 3.314111f0
ca = 0.07
cl = 3.3273785f0
ca = 0.07
cl = 3.435521f0
ca = 0.07
cl = 3.7068956f0
ca = 0.09
cl = 3.663776f0
ca = 0.06
cl = 3.4952502f0
ca = 0.06
cl = 3.3658726f0
ca = 0.06
cl = 3.3188617f0


┌ Info: Epoch 3
└ @ Main C:\Users\Zubu\.julia\packages\Flux\oX9Pi\src\optimise\train.jl:99


ca = 0.06
cl = 3.3100328f0
ca = 0.06
cl = 3.3115735f0
ca = 0.06
cl = 3.3134732f0
ca = 0.06
cl = 3.3131578f0
ca = 0.07
cl = 3.3121734f0
ca = 0.09
cl = 3.3119862f0
ca = 0.06
cl = 3.316441f0
ca = 0.06
cl = 3.3287017f0
ca = 0.07
cl = 3.3454483f0
ca = 0.06
cl = 3.3763125f0


┌ Info: Epoch 4
└ @ Main C:\Users\Zubu\.julia\packages\Flux\oX9Pi\src\optimise\train.jl:99


ca = 0.08
cl = 3.4224858f0
ca = 0.06
cl = 3.495653f0
ca = 0.07
cl = 3.615797f0
ca = 0.08
cl = 3.7969654f0
ca = 0.08
cl = 3.7024605f0
ca = 0.06
cl = 3.608518f0
ca = 0.06
cl = 3.5169275f0
ca = 0.06
cl = 3.4308205f0
ca = 0.07
cl = 3.3695858f0
ca = 0.08
cl = 3.3448803f0


┌ Info: Epoch 5
└ @ Main C:\Users\Zubu\.julia\packages\Flux\oX9Pi\src\optimise\train.jl:99


ca = 0.08
cl = 3.3348541f0
ca = 0.07
cl = 3.3304186f0
ca = 0.07
cl = 3.3316631f0
ca = 0.06
cl = 3.3414712f0
ca = 0.06
cl = 3.3555548f0
ca = 0.06
cl = 3.3886023f0
ca = 0.05
cl = 3.474331f0
ca = 0.06
cl = 3.4434056f0
ca = 0.06
cl = 3.404199f0
ca = 0.07
cl = 3.4042158f0
